In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 P2") 
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

import os

22/09/07 11:01:04 WARN Utils: Your hostname, Janes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.93.119 instead (on interface en0)
22/09/07 11:01:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/07 11:01:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
os.getcwd()

'/Users/janevieren/Downloads/School Locations'

In [4]:
schools = spark.read.option("header", True).csv('/Users/janevieren/Downloads/School Locations/SL')
schools.agg(*(countDistinct(col(c)).alias(c) for c in schools.columns))

Education_Sector,Entity_Type,SCHOOL_NO,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,Address_Postcode,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
3,2,2154,2111,6,1,2333,13,1328,1,581,2166,14,1076,1,585,2312,80,80,2330,2330


In [11]:
# only get school name, address line, town, state, postcode, x and y
schools_rel = schools.select("School_Name", "Address_Line_1", "Address_Town", "Address_State", "Address_Postcode")
schools_rel # assume people don't fill in address line 2 as most are null

School_Name,Address_Line_1,Address_Town,Address_State,Address_Postcode
Alberton Primary ...,21 Thomson Street,Alberton,VIC,3971
Allansford and Di...,Frank Street,Allansford,VIC,3277
Avoca Primary School,118 Barnett Street,Avoca,VIC,3467
Avenel Primary Sc...,40 Anderson Street,Avenel,VIC,3664
Warrandyte Primar...,5-11 Forbes Street,Warrandyte,VIC,3113
Belmont Primary S...,51 Mt Pleasant Road,Belmont,VIC,3216
Bacchus Marsh Pri...,56-62 Lerderderg ...,Bacchus Marsh,VIC,3340
Ballarat Primary ...,401 Dana Street,Ballarat,VIC,3350
Wodonga Primary S...,Brockley Street,Wodonga,VIC,3690
Berwick Primary S...,37 Fairholme Boul...,Berwick,VIC,3806


In [13]:
schools_nodup = schools_rel.dropDuplicates()

# save as csv
schools_nodup.write.csv("/Users/janevieren/Downloads/School Locations/school_vic_clean.csv")